# Desafio PTC Group 
Esse ntb faz parte do desafio enviado por email e tem como objetivo inserir e ler dados do DBFS que são oriundos do site Kaggle. 
São dados relacionados a vendas de um supermercado. Link https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales

In [0]:
from pyspark.sql.functions import *
import datetime

In [0]:
# Pode ser utilizado para inclusão no arquivo para evidênciar a data da carga
data_carga = datetime.date.today()

# Variaeveis do path Origem
file_location = "/FileStore/tables/supermarket_sales.csv"
file_type = "csv"

# Variavel do path Destino
file_save = "/FileStore/tables/"

# Inferindo schema com delimitador do arquivo
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Aplicando opções para ler o arquivo CSV. Para outros tipos de arquivo, necessário especificar.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


In [0]:
# Não foi necessário efetuar o ajuste do schema de dados. Apesar de não existir colunas nulas, esta setado aceitando essa condição.
df.printSchema()

# Verificando a quantidade de registros unicos
df.distinct().count()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

Out[45]: 1000

In [0]:
df = df.withColumnRenamed("Invoice ID", "Invoice_ID").withColumnRenamed("Customer type", "Customer_type")\
    .withColumnRenamed("Unit price", "Unit_price").withColumnRenamed("Tax 5%", "Tax_5_Perc")\
        .withColumnRenamed("Product line", "Product_line").withColumnRenamed("cogs", "Cogs")\
            .withColumnRenamed("gross margin percentage", "Gross_margin_percentage")\
                .withColumnRenamed("gross income", "Gross_income")

In [0]:
df = df.withColumn("Tax_5_Perc", round(df.Tax_5_Perc, 2)).withColumn("Total", round(df.Total, 2))\
    .withColumn("Cogs", round(df.Cogs, 2)).withColumn("Gross_margin_percentage", round(df.Gross_margin_percentage, 2))\
        .withColumn("Gross_income", round(df.Gross_income, 2))


In [0]:
df = df.withColumn("Total_Sem_Taxa", round(col("Unit_price") * col("Quantity"), 2))

In [0]:
# Efetuando o agrupamento por duas colunas, arredondando a coluna Total_Sem_Taxa e Classificando pela coluna Branch

df_agrupado = df.groupBy("Branch", "Payment")\
    .agg(round(sum("Total_Sem_Taxa"), 2).alias("Total_Vendas")).sort(col("Branch"))


In [0]:
# Criando uma nova coluna 'Categoria' com base na coluna Unit_Price
df_categorizado = df.withColumn("Categoria",
                                when(col("Unit_price") >= 90, "Classe_A")
                                .when((col("Unit_price") >= 40) & (col("Unit_price") < 90), "Classe_B")
                                .when((col("Unit_price") > 10) & (col("Unit_price") < 40), "Classe_C")
                                .otherwise("Classe_D"))

In [0]:
# Adicionando colunas de Dia, Mes e Ano para facilitar na gravação dos dados de forma particionada
df = df.withColumn("Year", year("Date")) \
                           .withColumn("Month", month("Date")) \
                           .withColumn("Day", dayofmonth("Date"))


# Também é possivel incluir uma coluna de controle relacionado a data da carga. Nesse caso não achei necessário pois o conjunto de dados possui informações de data.

# df = df.withColumn("DataCarga", current_date())

In [0]:
# Inserindo o nome e numero do mês para a classificação da legenda em ordem de mês no grafico do total de vendas por mês

df = df.withColumn("NomeMes",
                    when(col("Month") == 1, "01. Jan")
                    .when(col("Month") == 2, "02. Fev")
                    .when(col("Month") == 3, "03. Mar")
                    .when(col("Month") == 4, "04. Abr")
                    .when(col("Month") == 5, "05. Mai")
                    .when(col("Month") == 6, "06. Jun")
                    .when(col("Month") == 7, "07. Jul")
                    .when(col("Month") == 8, "08. Ago")
                    .when(col("Month") == 9, "09. Set")
                    .when(col("Month") == 10, "10. Out")
                    .when(col("Month") == 11, "11. Nov")
                    .when(col("Month") == 12, "12. Dez")
                    .otherwise("Outros"))

In [0]:
# Visualizando dataFrame ordenaod pela coluna NomeMes
df.sort(col("NomeMes")).display()

Invoice_ID,Branch,City,Customer_type,Gender,Product_line,Unit_price,Quantity,Tax_5_Perc,Total,Date,Time,Payment,Cogs,Gross_margin_percentage,Gross_income,Rating,Total_Sem_Taxa,Year,Month,Day,NomeMes
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.14,548.97,2019-01-05,2024-05-01T13:08:00.000+0000,Ewallet,522.83,4.76,26.14,9.1,522.83,2019,1,5,01. Jan
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.29,489.05,2019-01-27,2024-05-01T20:33:00.000+0000,Ewallet,465.76,4.76,23.29,8.4,465.76,2019,1,27,01. Jan
665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.63,76.15,2019-01-10,2024-05-01T17:15:00.000+0000,Credit card,72.52,4.76,3.63,7.2,72.52,2019,1,10,01. Jan
299-46-1805,B,Mandalay,Member,Female,Sports and travel,93.72,6,28.12,590.44,2019-01-15,2024-05-01T16:19:00.000+0000,Cash,562.32,4.76,28.12,4.5,562.32,2019,1,15,01. Jan
765-26-6951,A,Yangon,Normal,Male,Sports and travel,72.61,6,21.78,457.44,2019-01-01,2024-05-01T10:39:00.000+0000,Credit card,435.66,4.76,21.78,6.9,435.66,2019,1,1,01. Jan
329-62-1586,A,Yangon,Normal,Male,Food and beverages,54.67,3,8.2,172.21,2019-01-21,2024-05-01T18:00:00.000+0000,Credit card,164.01,4.76,8.2,8.6,164.01,2019,1,21,01. Jan
145-94-9061,B,Mandalay,Normal,Female,Food and beverages,88.36,5,22.09,463.89,2019-01-25,2024-05-01T19:48:00.000+0000,Cash,441.8,4.76,22.09,9.6,441.8,2019,1,25,01. Jan
149-71-6266,B,Mandalay,Member,Male,Sports and travel,78.07,9,35.13,737.76,2019-01-28,2024-05-01T12:43:00.000+0000,Cash,702.63,4.76,35.13,4.5,702.63,2019,1,28,01. Jan
640-49-2076,B,Mandalay,Normal,Male,Sports and travel,83.78,8,33.51,703.75,2019-01-10,2024-05-01T14:49:00.000+0000,Cash,670.24,4.76,33.51,5.1,670.24,2019,1,10,01. Jan
232-16-2483,C,Naypyitaw,Member,Female,Sports and travel,68.12,1,3.41,71.53,2019-01-07,2024-05-01T12:28:00.000+0000,Ewallet,68.12,4.76,3.41,6.8,68.12,2019,1,7,01. Jan


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Utilizando o 'for' para efetuar a gravação de dados em ambos os formatos .parquet e .delta. 

list_format = ["parquet", "delta"]

for i in list_format:
    df.write.format(f"{i}").partitionBy('Year','Month','Day').option("overwriteSchema", "true").mode("overwrite").save(f'{file_save}/{i}')

In [0]:
# Os arquivos foram gravados de forma particionada por Ano, Mes e Dia. Dependendo da situação pode ser inserido um filtro considerando essas colunas para melhor desempenho.

dbfs_parquet = 'dbfs:/FileStore/tables/parquet'
df_parquet = spark.read.format('parquet').load(dbfs_parquet)
df_parquet.display()

Invoice_ID,Branch,City,Customer_type,Gender,Product_line,Unit_price,Quantity,Tax_5_Perc,Total,Date,Time,Payment,Cogs,Gross_margin_percentage,Gross_income,Rating,Total_Sem_Taxa,NomeMes,Year,Month,Day
252-56-2699,A,Yangon,Normal,Male,Food and beverages,43.19,10,21.6,453.5,2019-02-07,2024-05-01T16:48:00.000+0000,Ewallet,431.9,4.76,21.6,8.2,431.9,02. Fev,2019,2,7
635-40-6220,A,Yangon,Normal,Male,Health and beauty,89.6,8,35.84,752.64,2019-02-07,2024-05-01T11:28:00.000+0000,Ewallet,716.8,4.76,35.84,6.6,716.8,02. Fev,2019,2,7
214-17-6927,C,Naypyitaw,Normal,Female,Food and beverages,16.48,6,4.94,103.82,2019-02-07,2024-05-01T18:23:00.000+0000,Ewallet,98.88,4.76,4.94,9.9,98.88,02. Fev,2019,2,7
380-94-4661,C,Naypyitaw,Member,Male,Electronic accessories,65.94,4,13.19,276.95,2019-02-07,2024-05-01T13:05:00.000+0000,Credit card,263.76,4.76,13.19,6.9,263.76,02. Fev,2019,2,7
868-81-1752,B,Mandalay,Normal,Male,Home and lifestyle,22.02,9,9.91,208.09,2019-02-07,2024-05-01T18:48:00.000+0000,Cash,198.18,4.76,9.91,6.8,198.18,02. Fev,2019,2,7
810-60-6344,C,Naypyitaw,Normal,Female,Electronic accessories,40.86,8,16.34,343.22,2019-02-07,2024-05-01T14:38:00.000+0000,Credit card,326.88,4.76,16.34,6.5,326.88,02. Fev,2019,2,7
533-33-5337,B,Mandalay,Normal,Male,Electronic accessories,79.39,10,39.7,833.6,2019-02-07,2024-05-01T20:24:00.000+0000,Cash,793.9,4.76,39.7,6.2,793.9,02. Fev,2019,2,7
827-44-5872,B,Mandalay,Member,Female,Food and beverages,54.36,10,27.18,570.78,2019-02-07,2024-05-01T11:28:00.000+0000,Credit card,543.6,4.76,27.18,6.1,543.6,02. Fev,2019,2,7
583-41-4548,C,Naypyitaw,Normal,Male,Home and lifestyle,16.67,7,5.83,122.52,2019-02-07,2024-05-01T11:36:00.000+0000,Ewallet,116.69,4.76,5.83,7.4,116.69,02. Fev,2019,2,7
560-49-6611,A,Yangon,Member,Female,Sports and travel,45.58,1,2.28,47.86,2019-02-07,2024-05-01T14:13:00.000+0000,Cash,45.58,4.76,2.28,9.8,45.58,02. Fev,2019,2,7


In [0]:
dbfs_delta = 'dbfs:/FileStore/tables/delta'
df_delta = spark.read.format('delta').load(dbfs_delta)
df_delta.display()

Invoice_ID,Branch,City,Customer_type,Gender,Product_line,Unit_price,Quantity,Tax_5_Perc,Total,Date,Time,Payment,Cogs,Gross_margin_percentage,Gross_income,Rating,Total_Sem_Taxa,Year,Month,Day,NomeMes
252-56-2699,A,Yangon,Normal,Male,Food and beverages,43.19,10,21.6,453.5,2019-02-07,2024-05-01T16:48:00.000+0000,Ewallet,431.9,4.76,21.6,8.2,431.9,2019,2,7,02. Fev
635-40-6220,A,Yangon,Normal,Male,Health and beauty,89.6,8,35.84,752.64,2019-02-07,2024-05-01T11:28:00.000+0000,Ewallet,716.8,4.76,35.84,6.6,716.8,2019,2,7,02. Fev
214-17-6927,C,Naypyitaw,Normal,Female,Food and beverages,16.48,6,4.94,103.82,2019-02-07,2024-05-01T18:23:00.000+0000,Ewallet,98.88,4.76,4.94,9.9,98.88,2019,2,7,02. Fev
380-94-4661,C,Naypyitaw,Member,Male,Electronic accessories,65.94,4,13.19,276.95,2019-02-07,2024-05-01T13:05:00.000+0000,Credit card,263.76,4.76,13.19,6.9,263.76,2019,2,7,02. Fev
868-81-1752,B,Mandalay,Normal,Male,Home and lifestyle,22.02,9,9.91,208.09,2019-02-07,2024-05-01T18:48:00.000+0000,Cash,198.18,4.76,9.91,6.8,198.18,2019,2,7,02. Fev
810-60-6344,C,Naypyitaw,Normal,Female,Electronic accessories,40.86,8,16.34,343.22,2019-02-07,2024-05-01T14:38:00.000+0000,Credit card,326.88,4.76,16.34,6.5,326.88,2019,2,7,02. Fev
533-33-5337,B,Mandalay,Normal,Male,Electronic accessories,79.39,10,39.7,833.6,2019-02-07,2024-05-01T20:24:00.000+0000,Cash,793.9,4.76,39.7,6.2,793.9,2019,2,7,02. Fev
827-44-5872,B,Mandalay,Member,Female,Food and beverages,54.36,10,27.18,570.78,2019-02-07,2024-05-01T11:28:00.000+0000,Credit card,543.6,4.76,27.18,6.1,543.6,2019,2,7,02. Fev
583-41-4548,C,Naypyitaw,Normal,Male,Home and lifestyle,16.67,7,5.83,122.52,2019-02-07,2024-05-01T11:36:00.000+0000,Ewallet,116.69,4.76,5.83,7.4,116.69,2019,2,7,02. Fev
560-49-6611,A,Yangon,Member,Female,Sports and travel,45.58,1,2.28,47.86,2019-02-07,2024-05-01T14:13:00.000+0000,Cash,45.58,4.76,2.28,9.8,45.58,2019,2,7,02. Fev
